In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
ATLAS_URI = os.getenv('ATLAS_URI')

if not (ATLAS_URI or EDEN_API_KEY):
    raise Exception ("'ATLAS_URI' or 'EDEN_API_KEY' is not set. Please set it above to continue...")
print("Fetched Env. Variables")

Fetched Env. Variables


In [3]:
DB_NAME = 'stackup_zendesk'
COLLECTION_NAME = 'data'
INDEX_NAME = 'idx_embedding'

In [4]:
import pymongo

mongodb_client = pymongo.MongoClient(ATLAS_URI)
print ("Atlas client initialized")

Atlas client initialized


In [5]:
from langchain_community.embeddings.edenai import EdenAiEmbeddings

embed_model = EdenAiEmbeddings(edenai_api_key=os.getenv("EDEN_API_KEY"), provider="openai", model="1536__text-embedding-ada-002")

In [6]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_8872/1756614690.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=None)


LLM is explicitly disabled. Using MockLLM.


In [7]:
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from llama_index.core import StorageContext

vector_store = MongoDBAtlasVectorSearch(
    mongodb_client = mongodb_client,
    db_name = DB_NAME, collection_name = COLLECTION_NAME,
    index_name  = 'idx_embeddings',
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [8]:
%%time
from llama_index.core import SimpleDirectoryReader

data_dir = 'data'

docs = SimpleDirectoryReader(
        input_dir=data_dir
).load_data()

print (f"Loaded {len(docs)} chunks from '{data_dir}'")

Loaded 1 chunks from 'data'
CPU times: user 9.08 ms, sys: 4.33 ms, total: 13.4 ms
Wall time: 17.5 ms


In [9]:
%%time
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    docs, 
    storage_context=storage_context,
    service_context=service_context,
)

CPU times: user 197 ms, sys: 0 ns, total: 197 ms
Wall time: 11.4 s
